# Get References from PDFs 

This guide shows you how to use LlamaIndex to get in-line page number citations in the response (and the response is streamed).

This is a simple combination of using the page number metadata in our PDF loader along with our indexing/query abstractions to use this information.

<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/citation/pdf_page_reference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index-llms-openai

In [ ]:
!pip install llama-index

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    download_loader,
    RAKEKeywordTableIndex,
)

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")

Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

Load document and build index

In [ ]:
reader = SimpleDirectoryReader(input_files=["./data/10k/lyft_2021.pdf"])
data = reader.load_data()

In [ ]:
index = VectorStoreIndex.from_documents(data)

In [ ]:
query_engine = index.as_query_engine(streaming=True, similarity_top_k=3)

Stream response with page citation

In [ ]:
response = query_engine.query(
    "What was the impact of COVID? Show statements in bullet form and show"
    " page reference after each statement."
)
response.print_response_stream()


• The ongoing COVID-19 pandemic continues to impact communities in the United States, Canada and globally (page 6). 
• The pandemic and related responses caused decreased demand for our platform leading to decreased revenues as well as decreased earning opportunities for drivers on our platform (page 6).
• Our business continues to be impacted by the COVID-19 pandemic (page 6).
• The exact timing and pace of the recovery remain uncertain (page 6).
• The extent to which our operations will continue to be impacted by the pandemic will depend largely on future developments, which are highly uncertain and cannot be accurately predicted (page 6).
• An increase in cases due to variants of the virus has caused many businesses to delay employees returning to the office (page 6).
• We anticipate that continued social distancing, altered consumer behavior, reduced travel and commuting, and expected corporate cost cutting will be significant challenges for us (page 6).
• We have adopted multiple

Inspect source nodes

In [ ]:
for node in response.source_nodes:
    print("-----")
    text_fmt = node.node.get_content().strip().replace("\n", " ")[:1000]
    print(f"Text:\t {text_fmt} ...")
    print(f"Metadata:\t {node.node.metadata}")
    print(f"Score:\t {node.score:.3f}")

-----
Text:	 Impact of COVID-19 to our BusinessThe  ongoing  COVID-19  pandemic  continues  to  impact  communities  in  the  United  States,  Canada  and  globally.  Since  the  pandemic  began  in  March  2020,governments  and  private  businesses  -  at  the  recommendation  of  public  health  officials  -  have  enacted  precautions  to  mitigate  the  spread  of  the  virus,  including  travelrestrictions  and  social  distancing  measures  in  many  regions  of  the  United  States  and  Canada,  and  many  enterprises  have  instituted  and  maintained  work  from  homeprograms and limited  the number of employees on site. Beginning in the middle of March 2020, the pandemic and these related responses caused decreased demand for ourplatform  leading to decreased revenues as well as decreased earning opportunities for drivers on our platform. Our business continues to be impacted by the COVID-19pandemic. Although  we have seen some signs of demand improving, particularly compare